In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%matplotlib inline
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

os.chdir('/content/drive/My Drive/GitHub Repositories/Tuberculosis/Tuberculosis Classification')

trainDir = './data/Train'
validDir = './data/Test'

IMG_SIZE = 512 # slightly smaller than vgg16 normally expects
datagen = ImageDataGenerator(rescale=1./255, 
                             samplewise_center=False, 
                             samplewise_std_normalization=False, 
                             horizontal_flip = True, 
                             vertical_flip = False, 
                             height_shift_range = 0.15, 
                             width_shift_range = 0.15, 
                             rotation_range = 5, 
                             shear_range = 0.01,
                             fill_mode = 'nearest',
                             zoom_range=0.1,  
                             )

train_gen = datagen.flow_from_directory(
    directory=trainDir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=2019)

valid_gen = datagen.flow_from_directory(
    directory=validDir,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    seed=2019)




Using TensorFlow backend.


Found 721 images belonging to 2 classes.
Found 79 images belonging to 2 classes.


In [3]:
t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(2, 4, figsize = (16, 8))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = 0, vmax = 1)
    c_ax.set_title('%s' % ('Pulmonary Abnormality' if np.argmax(c_y)>0.5 else 'Healthy'))
    c_ax.axis('off')

NameError: ignored

In [4]:
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import *
from keras.optimizers import *

num_classes = 2
img_shape = (IMG_SIZE, IMG_SIZE, 3)
inputs = Input(img_shape)

vgg16 = VGG16(input_shape =  img_shape, include_top = False, weights = 'imagenet')
vgg16.trainable = False
vgg16_depth = vgg16.get_output_shape_at(0)[-1]
vgg16_outputs = vgg16(inputs)
bn_outputs = BatchNormalization(name='BatchNormalization')(vgg16_outputs)

# here we do an attention mechanism to turn pixels in the GAP on an off
atten_outputs = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(bn_outputs)
atten_outputs = Conv2D(32, kernel_size = (1,1), padding = 'same', activation = 'relu')(atten_outputs)
atten_outputs = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(atten_outputs)
atten_outputs = Conv2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(atten_outputs)

# fan it out to all of the channels
weights = np.ones((1, 1, 1, vgg16_depth))
outputs_up_c2 = Conv2D(vgg16_depth, kernel_size = (1,1), padding = 'same', activation = 'linear', use_bias = False, weights = [weights])
outputs_up_c2.trainable = False
atten_outputs = outputs_up_c2(atten_outputs)

multiply_outputs = multiply([atten_outputs, bn_outputs], name='Multiply')
gap_features_ouputs = GlobalAveragePooling2D()(multiply_outputs)
gap_mask_ouputs = GlobalAveragePooling2D()(atten_outputs)

# to account for missing values from the attention model
outputs = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features_ouputs, gap_mask_ouputs])
outputs = Dropout(0.5)(outputs)
outputs = Dense(128, activation = 'elu')(outputs)
outputs = Dropout(0.25)(outputs)
outputs = Dense(num_classes, activation = 'softmax')(outputs)

model = Model(inputs = [inputs], outputs = [outputs])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()





58892288/58889256 [==============================] - 6s 0us/step








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   (None, 16, 16, 512)  14714688    input_1[0][0]                    
__________________________________________________________________________________________________
BatchNormalization (BatchNormal (None, 16, 16, 512)  2048        vgg16[1][0]                      
________________________________________________________________________________________________

In [0]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

weight_path=os.path.join( './Fine Tuning with AttentionBasedVGG16', '{}_pretrainedattentionbased_vgg16model.bestweights.hdf5'.format('TB'))
checkpoint = ModelCheckpoint(weight_path, monitor='val_acc', verbose=1, save_best_only=True, mode='auto', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="'val_acc'", mode="auto", patience=30) 

callbacks_list = [checkpoint, early, reduceLROnPlat]

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size
model.fit_generator(train_gen, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_gen, validation_steps=STEP_SIZE_VALID,
                    epochs = 100, callbacks = callbacks_list)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '




Epoch 1/100


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


 2/22 [=>............................] - ETA: 12:42 - loss: 0.5729 - acc: 0.6719

...

10/22 [============>.................] - ETA: 6:24 - loss: 0.6054 - acc: 0.6937

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 15 could not be retrieved. It could be because a worker has died.
  UserWarning)


15/22 [===================>..........] - ETA: 3:26 - loss: 0.5944 - acc: 0.7075

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:610: UserWarning: The input 5 could not be retrieved. It could be because a worker has died.
  UserWarning)


19/22 [========================>.....] - ETA: 1:29 - loss: 0.5746 - acc: 0.7197